In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv('../data/Loan_Default.csv')
df.describe(include="all")

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
count,148670.000000,148670.0,145326,148670,147762,148670,148536,148670,148670,148670,...,148670,148670.000000,148670,148470,148470,133572.000000,148670,148670,148670.000000,124549.000000
unique,NaN,NaN,2,4,2,3,4,2,2,2,...,4,NaN,2,7,2,NaN,4,2,NaN,NaN
top,NaN,NaN,cf,Male,nopre,type1,p3,l1,nopc,nob/c,...,CIB,NaN,CIB,45-54,to_inst,NaN,North,direct,NaN,NaN
freq,NaN,NaN,135348,42346,124621,113173,55934,142344,148114,127908,...,48152,NaN,74392,34720,95814,NaN,74722,148637,NaN,NaN
mean,99224.500000,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,699.789103,NaN,NaN,NaN,72.746457,NaN,NaN,0.246445,37.732932
std,42917.476598,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,115.875857,NaN,NaN,NaN,39.967603,NaN,NaN,0.430942,10.545435
min,24890.000000,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,500.000000,NaN,NaN,NaN,0.967478,NaN,NaN,0.000000,5.000000
25%,62057.250000,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,599.000000,NaN,NaN,NaN,60.474860,NaN,NaN,0.000000,31.000000
50%,99224.500000,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,699.000000,NaN,NaN,NaN,75.135870,NaN,NaN,0.000000,39.000000
75%,136391.750000,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,800.000000,NaN,NaN,NaN,86.184211,NaN,NaN,0.000000,45.000000


In [9]:
y = df["Status"]
X = df.drop(["Status", "ID"], axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
num_attribs = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]
cat_attribs = [col for col in X_train.columns if X_train[col].dtype == 'object']
ordinal_attribs = ["age", "total_units"]
nominal_attribs = [x for x in cat_attribs if x not in ordinal_attribs]

print(num_attribs)
print(ordinal_attribs)
print(nominal_attribs)

['year', 'loan_amount', 'rate_of_interest', 'Interest_rate_spread', 'Upfront_charges', 'term', 'property_value', 'income', 'Credit_Score', 'LTV', 'dtir1']
['age', 'total_units']
['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose', 'Credit_Worthiness', 'open_credit', 'business_or_commercial', 'Neg_ammortization', 'interest_only', 'lump_sum_payment', 'construction_type', 'occupancy_type', 'Secured_by', 'credit_type', 'co-applicant_credit_type', 'submission_of_application', 'Region', 'Security_Type']


"age" and "total_units" are ordered categories, so OrdinalEncoder is appropriate
The rest of the categorical attributes are not ordered, so one-hot encoder is appropriate

In [12]:
numerical_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

nominal_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

ordinal_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(categories=[['<25', '25-34', '35-44', '45-54', '55-64', '65-74', '>74'], ['1U', '2U', '3U', '4U']])
)

In [13]:
preprocessing = ColumnTransformer([
("numerical", numerical_pipeline, num_attribs),
("nominal", nominal_pipeline, nominal_attribs),
("ordinal", ordinal_pipeline, ordinal_attribs)
])

In [19]:
X_train_prepared = preprocessing.fit_transform(X_train)
X_test_prepared = preprocessing.transform(X_test)

#conversion to data frame
X_train_prepared = pd.DataFrame(
    X_train_prepared,
    columns=preprocessing.get_feature_names_out(),
    index=X_train.index)

X_test_prepared = pd.DataFrame(
    X_test_prepared,
    columns=preprocessing.get_feature_names_out(),
    index=X_test.index)

In [15]:
X_train_prepared.describe(include="all")

,numerical__year,numerical__loan_amount,numerical__rate_of_interest,numerical__Interest_rate_spread,numerical__Upfront_charges,numerical__term,numerical__property_value,numerical__income,numerical__Credit_Score,numerical__LTV,...,nominal__submission_of_application_not_inst,nominal__submission_of_application_to_inst,nominal__Region_North,nominal__Region_North-East,nominal__Region_central,nominal__Region_south,nominal__Security_Type_Indriect,nominal__Security_Type_direct,ordinal__age,ordinal__total_units
count,118936.0,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,...,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000
mean,0.0,5.705323e-18,-7.305801e-16,8.038233e-17,2.067059e-16,-4.571428e-16,-4.939137e-17,-5.693375e-17,3.718317e-16,1.653050e-16,...,0.353736,0.646264,0.502110,0.008399,0.058452,0.431039,0.000210,0.999790,3.137830,0.021398
std,0.0,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,...,0.478131,0.478131,0.499998,0.091263,0.234596,0.495224,0.014497,0.014497,1.419215,0.196632
min,0.0,-1.707940e+00,-8.249927e+00,-9.118744e+00,-1.089982e+00,-4.098103e+00,-1.401694e+00,-1.061997e+00,-1.724327e+00,-1.740993e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,-7.317496e-01,-5.783249e-01,-5.534866e-01,-6.445091e-01,4.253860e-01,-5.871160e-01,-4.707358e-01,-8.704210e-01,-2.360485e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000
50%,0.0,-1.894214e-01,-8.734241e-02,-8.713045e-02,-1.649559e-01,4.253860e-01,-2.089189e-01,-1.751051e-01,-7.889520e-03,5.062967e-02,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000
75%,0.0,5.698381e-01,4.445553e-01,4.278792e-01,2.927654e-01,4.253860e-01,3.147385e-01,2.129101e-01,8.632672e-01,2.831916e-01,...,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,4.000000,0.000000
max,0.0,1.759894e+01,8.116157e+00,6.564722e+00,2.029271e+01,4.253860e-01,4.660024e+01,8.802446e+01,1.725799e+00,1.874085e+02,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,3.000000


In [16]:
X_train_prepared.isnull().sum()

numerical__year                    0
numerical__loan_amount             0
numerical__rate_of_interest        0
numerical__Interest_rate_spread    0
numerical__Upfront_charges         0
                                  ..
nominal__Region_south              0
nominal__Security_Type_Indriect    0
nominal__Security_Type_direct      0
ordinal__age                       0
ordinal__total_units               0
Length: 61, dtype: int64